In [2]:
%matplotlib inline
import torch
import random
# from d2l import torch as d2l

In [3]:
# 生成数据集
def synthetic_data(w, b, num_examples): # 模型为 y=Xw+b+q(噪声) #@save
    X = torch.normal(0, 1, (num_examples, len(w))) # 数据集为1000行2列
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(-1, 1)

given_w = torch.tensor([2, -3.4]) # 给定线性参数(2，-3.4)
given_b = 4.2 # 给定参数4.2
features, labels = synthetic_data(given_w, given_b, 1000)
print('features:',features[0],'\nlabels:',labels[0])
#print(type(features[0]))

features: tensor([0.6762, 0.0300]) 
labels: tensor([5.4613])


In [3]:
# 读取数据集
def data_read(batch_size, features, labels): # 生成小批量数据表
    num_examples = len(features)
    indices = list(range(num_examples)) # 创建列表
    random.shuffle(indices) # 随机读取列表中的样本
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)]) 
        yield features[batch_indices], labels[batch_indices]
        
batch_size = 10

for X, y in data_read(batch_size, features, labels):
    print(X, '\n', y)
    break


tensor([[-0.4001,  1.9823],
        [ 0.4067, -0.9114],
        [-0.9392, -0.1528],
        [-1.3112,  0.4308],
        [ 1.1068,  1.3974],
        [ 1.8488,  0.5349],
        [-0.0821,  1.2399],
        [ 0.8343, -1.9747],
        [-1.0795,  1.7305],
        [ 2.4195,  1.3710]]) 
 tensor([[-3.3503],
        [ 8.1107],
        [ 2.8516],
        [ 0.1178],
        [ 1.6612],
        [ 6.0815],
        [-0.1747],
        [12.6018],
        [-3.8434],
        [ 4.3805]])


In [13]:
# 初始化模型参数
# 更新参数，并用自动微分来计算梯度
w = torch.normal(0, 0.01, size = (2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [14]:
# 定义模型
def linreg(X, w, b): #@save
    return torch.matmul(X, w) + b

In [15]:
# 定义损失函数
def squared_loss(y_hat, y): # 均方损失 #@save
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2 # 1/2(y1-y2)^2

In [16]:
# 定义优化算法
def sgd(params, lr, batch_size): # 小批量随机梯度下降 #@save
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            '''params.grad 其实是batch中所有样本的grad总和，
            所以这个时候除以batch_size就是相当于取一个平均值，
            这样就算下一次传入的batch_size改变了，最后也不会影响得到的平均数。'''
            param.grad.zero_() # 梯度置零

In [17]:
# 训练
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
for epoch in range(num_epochs):
    for X, y in data_read(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size) # 使用参数的梯度更新参数
    with torch.no_grad(): # 所有计算得出的tensor的requires_grad都自动设置为False
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1}, loss{float(train_l.mean()):f}')

epoch1, loss0.035415
epoch2, loss0.000122
epoch3, loss0.000049


In [ ]:
# 输出最终比较